In [64]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os
from IPython.core.display import display, HTML

C:\Users\nekna\AppData\Local\Temp\ipykernel_11828\2272534428.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# EDA

In [65]:
raw_path = '../data/raw/'
train = pd.read_csv(os.path.join(raw_path, 'train.csv'))
test = pd.read_csv(os.path.join(raw_path, 'test.csv'))

In [66]:
display(HTML("<h3>TRAIN DATASET</h3>"))
display(HTML(train.head().to_html()))
display(HTML("<h3>TEST DATASET</h3>"))
display(HTML(test.head().to_html()))

,ID,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,Сахарный диабет,Гепатит,Онкология,Хроническое заболевание легких,Бронжиальная астма,Туберкулез легких,ВИЧ/СПИД,Регулярный прим лекарственных средств,Травмы за год,Переломы,Статус Курения,Возраст курения,Сигарет в день,Пассивное курение,Частота пасс кур,Алкоголь,Возраст алког,Время засыпания,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы",ID_y,Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
0,54-102-358-02,М,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,0,0,0,0,0,0,0,0,0,0,Курит,15.0,20.0,0,NaN,употребляю в настоящее время,18.0,22:00:00,06:00:00,0,0,0,54-102-358-02,0,0,0,0,0
1,54-103-101-01,Ж,в разводе,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,0,1,0,0,0,0,0,0,1,0,1,Никогда не курил(а),NaN,NaN,0,NaN,никогда не употреблял,NaN,00:00:00,04:00:00,1,0,0,54-103-101-01,1,1,0,0,0
2,54-501-026-03,Ж,в браке в настоящее время,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,0,0,0,0,0,0,0,0,1,0,0,Никогда не курил(а),NaN,NaN,1,1-2 раза в неделю,употребляю в настоящее время,17.0,23:00:00,07:00:00,0,0,0,54-501-026-03,0,0,0,0,0
3,54-501-094-02,М,в браке в настоящее время,европейская,Русские,Атеист / агностик,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,0,0,1,0,0,0,0,1,0,0,Бросил(а),12.0,10.0,1,3-6 раз в неделю,употребляю в настоящее время,13.0,23:00:00,07:00:00,0,0,0,54-501-094-02,1,0,0,0,0
4,54-503-022-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,операторы и монтажники установок и машинного оборудования,0,0,1,1,1,0,0,0,0,0,1,0,1,Никогда не курил(а),NaN,NaN,1,не менее 1 раза в день,употребляю в настоящее время,16.0,23:00:00,06:00:00,0,0,0,54-503-022-01,1,0,1,1,0


,ID,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,Сахарный диабет,Гепатит,Онкология,Хроническое заболевание легких,Бронжиальная астма,Туберкулез легких,ВИЧ/СПИД,Регулярный прим лекарственных средств,Травмы за год,Переломы,Статус Курения,Возраст курения,Сигарет в день,Пассивное курение,Частота пасс кур,Алкоголь,Возраст алког,Время засыпания,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы"
0,54-001-019-01,Ж,вдовец / вдова,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,ремесленники и представители других отраслей промышленности,1,0,0,0,0,0,0,0,0,0,1,1,1,Никогда не курил(а),NaN,NaN,0,NaN,употребляю в настоящее время,23.0,21:30:00,05:00:00,0,1,1
1,54-002-133-01,Ж,в разводе,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,1,0,0,0,0,0,0,0,0,1,0,0,Никогда не курил(а),NaN,NaN,0,NaN,употребляю в настоящее время,22.0,00:00:00,09:00:00,0,0,0
2,54-001-007-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,1,0,0,0,0,0,0,0,0,1,0,0,Никогда не курил(а),NaN,NaN,0,NaN,употребляю в настоящее время,18.0,23:00:00,07:30:00,0,0,0
3,54-102-116-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,4 - профессиональное училище,низкоквалифицированные работники,1,0,0,0,1,0,0,0,0,0,0,0,1,Никогда не курил(а),NaN,NaN,0,NaN,никогда не употреблял,NaN,22:00:00,06:00:00,0,0,0
4,54-502-005-02,М,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,операторы и монтажники установок и машинного оборудования,0,1,0,0,0,0,0,0,0,0,1,1,1,Курит,14.0,20.0,0,NaN,употребляю в настоящее время,19.0,22:00:00,06:00:00,0,0,0


Как мы можем заметить, в столбцах возраст курения, сигарет в день и частота пассивного курения присутствуют пропуски. Избавимся от них следующим образом: Если человек никогда не курил, то возраст курения и сигарет в день приравниваем к нулю. Аналогично с пассивным курением.

In [67]:
(train['ID'] == train['ID_y']).all()

True

Также заметим, что id == id_y, поэтому мы можем просто отбросить этот столбец

In [68]:
def Preprocessing(df: pd.DataFrame) -> pd.DataFrame:
    if 'ID_y' in df.columns:
        df = df.drop('ID_y', axis=1)
    if 'ID' in df.columns:
        df = df.set_index('ID')
    #Избавляемся от пропусков
    df['Возраст курения'] = np.where(df['Статус Курения'] == 'Никогда не курил(а)', 0, df['Возраст курения'])
    df['Сигарет в день'] = np.where(df['Статус Курения'] == 'Никогда не курил(а)', 0, df['Сигарет в день'])
    df['Сигарет в день'] = np.where(df['Сигарет в день'].isna(), 0, df['Сигарет в день'])
    df['Частота пасс кур'] = np.where(df['Пассивное курение'] == 0, 'Ни разу в день', df['Частота пасс кур'])
    df['Возраст алког'] = np.where(df['Алкоголь'] == 'никогда не употреблял', 0, df['Возраст алког'])

    df = df.drop(df[df['Возраст алког'].isna()].index)
    df = df.drop(df[df['Пол'].isna()].index)
    df = df.drop(df[df['Частота пасс кур'].isna()].index)

    return df


In [69]:
train = Preprocessing(train)
test = test.set_index('ID')

Мы избавились от всех пропусков. Теперь уберем столбцы, которые мы будем предсказывать

In [70]:
clear_train = train.copy()
Target_columns = [
    'Артериальная гипертензия', 'ОНМК',
    'Стенокардия, ИБС, инфаркт миокарда',
    'Сердечная недостаточность', 
    'Прочие заболевания сердца'
]
target = clear_train[Target_columns]
clear_train = clear_train.drop(Target_columns, axis=1)
display(HTML(target.head().to_html()))
display(HTML(clear_train.head().to_html()))

,Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
ID,,,,,
54-102-358-02,0,0,0,0,0
54-103-101-01,1,1,0,0,0
54-501-026-03,0,0,0,0,0
54-501-094-02,1,0,0,0,0
54-503-022-01,1,0,1,1,0


,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,Сахарный диабет,Гепатит,Онкология,Хроническое заболевание легких,Бронжиальная астма,Туберкулез легких,ВИЧ/СПИД,Регулярный прим лекарственных средств,Травмы за год,Переломы,Статус Курения,Возраст курения,Сигарет в день,Пассивное курение,Частота пасс кур,Алкоголь,Возраст алког,Время засыпания,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы"
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
54-102-358-02,М,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,0,0,0,0,0,0,0,0,0,0,Курит,15.0,20.0,0,Ни разу в день,употребляю в настоящее время,18.0,22:00:00,06:00:00,0,0,0
54-103-101-01,Ж,в разводе,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,0,1,0,0,0,0,0,0,1,0,1,Никогда не курил(а),0.0,0.0,0,Ни разу в день,никогда не употреблял,0.0,00:00:00,04:00:00,1,0,0
54-501-026-03,Ж,в браке в настоящее время,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,0,0,0,0,0,0,0,0,1,0,0,Никогда не курил(а),0.0,0.0,1,1-2 раза в неделю,употребляю в настоящее время,17.0,23:00:00,07:00:00,0,0,0
54-501-094-02,М,в браке в настоящее время,европейская,Русские,Атеист / агностик,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,0,0,1,0,0,0,0,1,0,0,Бросил(а),12.0,10.0,1,3-6 раз в неделю,употребляю в настоящее время,13.0,23:00:00,07:00:00,0,0,0
54-503-022-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,операторы и монтажники установок и машинного оборудования,0,0,1,1,1,0,0,0,0,0,1,0,1,Никогда не курил(а),0.0,0.0,1,не менее 1 раза в день,употребляю в настоящее время,16.0,23:00:00,06:00:00,0,0,0


Разделим признаки на несколько списков для удобства

In [71]:
clear_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 950 entries, 54-102-358-02 to 54-702-016-01
Data columns (total 32 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Пол                                    950 non-null    object 
 1   Семья                                  950 non-null    object 
 2   Этнос                                  950 non-null    object 
 3   Национальность                         950 non-null    object 
 4   Религия                                950 non-null    object 
 5   Образование                            950 non-null    object 
 6   Профессия                              950 non-null    object 
 7   Вы работаете?                          950 non-null    int64  
 8   Выход на пенсию                        950 non-null    int64  
 9   Прекращение работы по болезни          950 non-null    int64  
 10  Сахарный диабет                        950 non-null    in

In [72]:
Categorial_columns = [
    'Пол', 'Семья', 'Этнос', 'Национальность', 'Религия', 'Образование', 
    'Профессия', 'Статус Курения', 'Частота пасс кур', 'Алкоголь',
    'Время засыпания', 'Время пробуждения'
]
Ohe_columns = [
    'Пол', 'Вы работаете?', 'Выход на пенсию', 'Прекращение работы по болезни', 'Сахарный диабет', 'Гепатит',
    'Онкология', 'Хроническое заболевание легких', 'Бронжиальная астма', 'Туберкулез легких ', 'ВИЧ/СПИД',
    'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы','Пассивное курение', 'Сон после обеда', 
    'Спорт, клубы', 'Религия, клубы'
]
Real_columns = ['Возраст курения', 'Сигарет в день', 'Возраст алког']
Education_col = 'Образование'
clear_train[Education_col] = clear_train[Education_col].str.slice(0,1).astype(np.int8).values
test[Education_col] = test[Education_col].str.slice(0,1).astype(np.int8).values


Ohe_int_columns = [Ohe_columns[i] for i in range(len(Ohe_columns)) if type(clear_train[Ohe_columns[i]][0]) == np.int64]
clear_train

,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,...,Сигарет в день,Пассивное курение,Частота пасс кур,Алкоголь,Возраст алког,Время засыпания,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы"
ID,,,,,,,,,,,,,,,,,,,,,
54-102-358-02,М,в браке в настоящее время,европейская,Русские,Христианство,3,низкоквалифицированные работники,1,0,0,...,20.0,0,Ни разу в день,употребляю в настоящее время,18.0,22:00:00,06:00:00,0,0,0
54-103-101-01,Ж,в разводе,европейская,Русские,Христианство,5,дипломированные специалисты,0,0,0,...,0.0,0,Ни разу в день,никогда не употреблял,0.0,00:00:00,04:00:00,1,0,0
54-501-026-03,Ж,в браке в настоящее время,европейская,Русские,Христианство,5,дипломированные специалисты,0,0,0,...,0.0,1,1-2 раза в неделю,употребляю в настоящее время,17.0,23:00:00,07:00:00,0,0,0
54-501-094-02,М,в браке в настоящее время,европейская,Русские,Атеист / агностик,3,низкоквалифицированные работники,1,0,0,...,10.0,1,3-6 раз в неделю,употребляю в настоящее время,13.0,23:00:00,07:00:00,0,0,0
54-503-022-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3,операторы и монтажники установок и машинного о...,0,0,1,...,0.0,1,не менее 1 раза в день,употребляю в настоящее время,16.0,23:00:00,06:00:00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54-103-006-01,М,в браке в настоящее время,европейская,Русские,Христианство,4,операторы и монтажники установок и машинного о...,1,0,0,...,20.0,0,Ни разу в день,употребляю в настоящее время,18.0,22:00:00,06:00:00,0,0,0
54-701-039-01,Ж,вдовец / вдова,европейская,Русские,Христианство,3,низкоквалифицированные работники,1,0,0,...,10.0,0,Ни разу в день,ранее употреблял,26.0,21:30:00,07:15:00,0,0,0
54-002-014-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3,служащие,0,1,0,...,10.0,0,Ни разу в день,употребляю в настоящее время,20.0,22:30:00,08:00:00,1,0,0


Преобразование типов

In [73]:
clear_train[Ohe_int_columns] = clear_train[Ohe_int_columns].astype(np.int8)
clear_train[Categorial_columns] = clear_train[Categorial_columns].astype('category')
clear_train[Real_columns] = clear_train[Real_columns].astype(np.float32)

In [74]:
clear_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 950 entries, 54-102-358-02 to 54-702-016-01
Data columns (total 32 columns):
 #   Column                                 Non-Null Count  Dtype   
---  ------                                 --------------  -----   
 0   Пол                                    950 non-null    category
 1   Семья                                  950 non-null    category
 2   Этнос                                  950 non-null    category
 3   Национальность                         950 non-null    category
 4   Религия                                950 non-null    category
 5   Образование                            950 non-null    category
 6   Профессия                              950 non-null    category
 7   Вы работаете?                          950 non-null    int8    
 8   Выход на пенсию                        950 non-null    int8    
 9   Прекращение работы по болезни          950 non-null    int8    
 10  Сахарный диабет                        950 no

In [75]:
display(HTML(train.head().to_html()))

,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,Сахарный диабет,Гепатит,Онкология,Хроническое заболевание легких,Бронжиальная астма,Туберкулез легких,ВИЧ/СПИД,Регулярный прим лекарственных средств,Травмы за год,Переломы,Статус Курения,Возраст курения,Сигарет в день,Пассивное курение,Частота пасс кур,Алкоголь,Возраст алког,Время засыпания,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы",Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
54-102-358-02,М,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,0,0,0,0,0,0,0,0,0,0,Курит,15.0,20.0,0,Ни разу в день,употребляю в настоящее время,18.0,22:00:00,06:00:00,0,0,0,0,0,0,0,0
54-103-101-01,Ж,в разводе,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,0,1,0,0,0,0,0,0,1,0,1,Никогда не курил(а),0.0,0.0,0,Ни разу в день,никогда не употреблял,0.0,00:00:00,04:00:00,1,0,0,1,1,0,0,0
54-501-026-03,Ж,в браке в настоящее время,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,0,0,0,0,0,0,0,0,1,0,0,Никогда не курил(а),0.0,0.0,1,1-2 раза в неделю,употребляю в настоящее время,17.0,23:00:00,07:00:00,0,0,0,0,0,0,0,0
54-501-094-02,М,в браке в настоящее время,европейская,Русские,Атеист / агностик,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,0,0,1,0,0,0,0,1,0,0,Бросил(а),12.0,10.0,1,3-6 раз в неделю,употребляю в настоящее время,13.0,23:00:00,07:00:00,0,0,0,1,0,0,0,0
54-503-022-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,операторы и монтажники установок и машинного оборудования,0,0,1,1,1,0,0,0,0,0,1,0,1,Никогда не курил(а),0.0,0.0,1,не менее 1 раза в день,употребляю в настоящее время,16.0,23:00:00,06:00:00,0,0,0,1,0,1,1,0


# MODELING

In [17]:
import os
import pandas as pd
import numpy as np

In [18]:
from sklearn.svm import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.compose import *
from sklearn.pipeline import *
from sklearn.metrics import *
from sklearn.impute import *
from sklearn.multioutput import *

In [19]:
import src.config as cfg

In [20]:
RS = 77

In [27]:
processed_data_path = 'data/processed/'
train = pd.read_pickle(os.path.join(processed_data_path, 'train.pkl'))
target = pd.read_pickle(os.path.join(processed_data_path, 'target.pkl'))

In [22]:
train[cfg.CAT_COLS] = train[cfg.CAT_COLS].astype('object')

In [23]:
# int8_cols = train.head(0).select_dtypes('int8').columns
# train[int8_cols] = train[int8_cols].astype(np.int32)
# target = target.astype(np.int32)

In [24]:
from functools import partial

In [25]:
scoring = partial(fbeta_score, beta=2.0)

In [28]:
train_idx, val_idx = train_test_split(
        train.index, test_size=0.2, random_state=7)

In [29]:
train_idx

Index(['54-501-041-01', '54-101-071-01', '54-102-244-01', '54-601-028-02',
       '54-601-006-01', '54-501-062-01', '54-001-070-02', '54-101-008-02',
       '54-002-111-01', '54-102-053-01',
       ...
       '54-002-004-02', '54-101-065-01', '54-501-038-01', '54-101-044-01',
       '54-103-069-01', '54-102-250-01', '54-701-012-02', '54-102-018-02',
       '54-102-029-01', '54-102-340-01'],
      dtype='object', name='ID', length=764)

In [32]:
val_idx.to_frame().to_pickle('temp.pkl')

In [33]:
import pickle

In [34]:
with open('tmp2.pkl', 'wb') as f:
    pickle.dump(val_idx, f)

In [36]:
with open('tmp2.pkl', 'rb') as f:
    val_idx2 = pickle.load(f)

In [39]:
type(train)

pandas.core.frame.DataFrame

pandas.core.indexes.base.Index

In [40]:
type(val_idx2)

pandas.core.indexes.base.Index

In [37]:
val_idx2

Index(['54-501-020-01', '54-101-047-01', '54-102-300-03', '54-103-046-01',
       '54-602-026-02', '54-002-118-01', '54-501-071-01', '54-501-055-02',
       '54-001-033-01', '54-001-062-01',
       ...
       '54-503-002-01', '54-601-019-02', '54-501-061-01', '54-503-008-01',
       '54-103-053-01', '54-504-003-01', '54-501-035-02', '54-102-159-01',
       '54-103-096-02', '54-701-005-01'],
      dtype='object', name='ID', length=191)

In [26]:
train_data, val_data, train_target, val_target = train_test_split(train, target, train_size=0.8, random_state=RS)

In [15]:
train_data.head()

,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,...,Сигарет в день,Пассивное курение,Частота пасс кур,Алкоголь,Возраст алког,Время засыпания,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы"
ID,,,,,,,,,,,,,,,,,,,,,
54-103-054-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,1,0,0,...,NaN,0,NaN,употребляю в настоящее время,26.0,22:00:00,06:00:00,0,1,0
54-102-299-01,Ж,вдовец / вдова,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,1,0,...,NaN,0,NaN,никогда не употреблял,NaN,22:30:00,08:30:00,0,1,0
54-102-138-01,Ж,никогда не был(а) в браке,европейская,Русские,Христианство,5 - ВУЗ,низкоквалифицированные работники,1,0,0,...,NaN,0,NaN,никогда не употреблял,NaN,22:00:00,06:30:00,0,0,0
54-601-033-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,4 - профессиональное училище,операторы и монтажники установок и машинного о...,0,1,0,...,NaN,0,NaN,употребляю в настоящее время,20.0,22:00:00,07:00:00,0,0,0
54-602-054-01,Ж,в разводе,европейская,Русские,Христианство,4 - профессиональное училище,ведение домашнего хозяйства,0,1,0,...,10.0,0,NaN,употребляю в настоящее время,18.0,00:00:00,07:00:00,1,0,0


In [16]:
real_pipe = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler())
]
)

In [17]:
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='NA')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

In [2]:
import category_encoders as ce

/opt/miniconda3/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [18]:
preprocess_pipe = ColumnTransformer(transformers=[
    ('real_cols', real_pipe, cfg.REAL_COLS),
    ('cat_cols', cat_pipe, cfg.CAT_COLS),
    ('woe_cat_cols', ce.WOEEncoder(), cfg.CAT_COLS),
    ('ohe_cols', 'passthrough', cfg.OHE_COLS)
]
)

In [19]:
model = LinearSVC()

In [20]:
model_pipe = Pipeline([
    ('preprocess', preprocess_pipe),
    ('model', model)
]
)

In [21]:
multiout_model_pipe = MultiOutputClassifier(model_pipe, n_jobs=4)

In [24]:
scores = cross_val_score(
    estimator=multiout_model_pipe,
    X=train_data,
    y=train_target,
    scoring='f1_samples',
    cv=3,
    n_jobs=1
)

/opt/miniconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/miniconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/miniconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/miniconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

In [25]:
scores

array([0.24287582, 0.21764706, 0.23425197])